In [1]:
# Code used is currently heavily based on a tutorial getting started with variational autoencoder using pytorch on Debugger Cafe
# This will be changed over time

In [2]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader

import torch.nn.functional as F

In [3]:
import os
import pandas as pd
import torch

from torch.utils.data import Dataset

from skimage import io
from PIL import Image

class ActiveVisionDataset (Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file, index_col=None)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        if type(index) == torch.Tensor:
            index = index.item()
        #Maybe this should be IMG?
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        #print(image.type())
        y_label = torch.tensor(int(self.annotations.iloc[index,1]))
        #cam_loc = torch.tensor(int(self.annotations.iloc[index,2]))
        
        if self.transform:
            image = self.transform(image)
        
        return image, y_label

In [4]:
# leanring parameters
epochs = 20
batch_size = 16
lr = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


kernel_size = 4
stride = 1
padding = 0
init_kernel = 16 # initial number of filters

cuda


In [5]:
os.chdir("imgs")

In [6]:
dataset = ActiveVisionDataset(csv_file='rgbCSV.csv', root_dir= 'rgbImg/', transform = torchvision.transforms.ToTensor())
train_data, val_data = torch.utils.data.random_split(dataset, [2400, 600])
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)


In [7]:
#for data in train_loader:
#    print(im_path)
#    print("Data: ", data)

In [8]:
#print(list(train_data))
#type(train_loader)
#print(train_data.shape)

#for i, (images, labels) in enumerate(train_loader):
#    print(images.shape)

In [9]:
# define a Conv VAE
class ConvVAE(nn.Module):
    def __init__(self):
        super(ConvVAE, self).__init__()
 
        # encoder
        self.enc1 = nn.Conv2d(
            in_channels=3, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc2 = nn.Conv2d(
            in_channels=init_kernel, out_channels=init_kernel*2, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc3 = nn.Conv2d(
            in_channels=init_kernel*2, out_channels=init_kernel*4, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc4 = nn.Conv2d(
            in_channels=init_kernel*4, out_channels=init_kernel*8, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc5 = nn.Conv2d(
            in_channels=init_kernel*8, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        # decoder 
        self.dec1 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=init_kernel*8, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec2 = nn.ConvTranspose2d(
            in_channels=init_kernel*8, out_channels=init_kernel*4, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec3 = nn.ConvTranspose2d(
            in_channels=init_kernel*4, out_channels=init_kernel*2, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec4 = nn.ConvTranspose2d(
            in_channels=init_kernel*2, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec5 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=3, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling
        return sample
 
    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        x = self.enc5(x)
        # get `mu` and `log_var`
        mu = x
        log_var = x
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
 
        # decoding
        x = F.relu(self.dec1(z))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        reconstruction = torch.sigmoid(self.dec5(x))
        return reconstruction, mu, log_var

In [10]:
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
#import model
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
matplotlib.style.use('ggplot')

In [11]:
model = ConvVAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss(reduction='sum')

In [12]:
def final_loss(bce_loss, mu, logvar):
    """
    This function will add the reconstruction loss (BCELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    BCE = bce_loss 
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [13]:
def fit(model, dataloader):
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        data, _ = data
        data = data.to(device)
        #data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(data)
        bce_loss = criterion(reconstruction, data)
        loss = final_loss(bce_loss, mu, logvar)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

In [14]:
def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, _ = data
            data = data.to(device)
            #data = data.view(data.size(0), -1)
            reconstruction, mu, logvar = model(data)
            bce_loss = criterion(reconstruction, data)
            loss = final_loss(bce_loss, mu, logvar)
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            if i == int(len(val_data)/dataloader.batch_size) - 1:
                num_rows = 8
                both = torch.cat((data.view(batch_size, 3, 224, 224)[:8], 
                                  reconstruction.view(batch_size, 3, 224, 224)[:8]))
                save_image(both.cpu(), f"../outputs/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [15]:
#print(model)

In [16]:
train_loss = []
val_loss = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} of {epochs}")
    train_epoch_loss = fit(model, train_loader)
    val_epoch_loss = validate(model, val_loader)
    train_loss.append(train_epoch_loss)
    val_loss.append(val_epoch_loss)
    print(f"Train Loss: {train_epoch_loss:.4f}")
    print(f"Val Loss: {val_epoch_loss:.4f}")

  0%|                                                                                          | 0/150 [00:00<?, ?it/s]

Epoch 1 of 20


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:41<00:00,  3.60it/s]
38it [00:04,  9.33it/s]                                                                                                
  1%|▌                                                                                 | 1/150 [00:00<00:15,  9.80it/s]

Train Loss: 82580.9688
Val Loss: 77953.3045
Epoch 2 of 20


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:40<00:00,  3.66it/s]
38it [00:04,  9.36it/s]                                                                                                
  1%|▌                                                                                 | 1/150 [00:00<00:15,  9.61it/s]

...

Train Loss: 72260.3762
Val Loss: 71890.7982
Epoch 19 of 20


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:41<00:00,  3.64it/s]
38it [00:04,  9.31it/s]                                                                                                
  1%|▌                                                                                 | 1/150 [00:00<00:15,  9.34it/s]

Train Loss: 72209.0188
Val Loss: 71861.4425
Epoch 20 of 20


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:41<00:00,  3.65it/s]
38it [00:04,  9.27it/s]                                                                                                

Train Loss: 72171.2778
Val Loss: 71811.5052


## Random stuff at the bottom

In [17]:
bebop

NameError: name 'bebop' is not defined

In [ ]:
import os
import pandas as pd
import torch

os.chdir("imgs")

In [ ]:
from torch.utils.data import Dataset

from skimage import io

root_dir= 'rgbImg/'
csv_file='rgbCSV.csv'

annotations = pd.read_csv('rgbCSV.csv', index_col=None)

img_path = os.path.join(root_dir, annotations.iloc[0, 0])
image = io.imread(img_path)
io.imshow(image)
image.shape

In [ ]:
dataset = ActiveVisionDataset(csv_file='rgbCSV.csv', root_dir= 'rgbImg/', transform = torchvision.transforms.ToTensor())

train_set, val_set = torch.utils.data.random_split(dataset, [80, 20])
train_loader = DataLoader(dataset=train_set, batch_size=4, shuffle=True)
val_loader = DataLoader(dataset=valt, batch_size=4, shuffle=True)